In [1]:
from ast import literal_eval
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

all_stopwords = set(stopwords.words('english') + list(string.punctuation))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/temilajumoke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/temilajumoke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data Preprocessing

In [3]:
CONTENT_ROOT_PATH = "../Entity_Recognition/Neighborhood_Separated_Articles"

In [4]:
data_file_name_dict = {
    '2014':'2014.csv',
    '2015':'2015.csv',
    '2016':'2016.csv',
    '2017':'2017.csv',
    '2018':'2018.csv',
}

In [5]:
file_name = data_file_name_dict['2014']
df = pd.read_csv(f'{CONTENT_ROOT_PATH}/{file_name}')
df = df.replace(to_replace ="('no article', 'no_id')", value="")
df.head(5)

,Unnamed: 0,dorchester,roxbury,mattapan,hyde_park,fenway,beacon_hill,downtown,south_boston,east_boston,...,charlestown,brighton,allston,west_end,roslindale,north_end,mission_hill,harbor_islands,longwood_medical_area,west_roxbury
0,0,('Before entering a South African prison in 19...,('n individuals who worked in finance insuran...,('The public scrap over who will serve as the ...,('The Boston Police Department has issued a co...,('Wahlburgers the burger joint owned by Dorch...,('Martin J Walsh becomes the 48th mayor of Bo...,('Last week Mayor Marty Walsh declared to the...,('The 20 year old who allegedly beat a disable...,('Saying that Massachusetts is facing “the big...,...,('Bright skies belie the early spring chill as...,('Boston’s new mayor Martin J Walsh has rep...,('A small army of city inspectors will soon be...,('Shakespeare’s “A Midsummer Night’s Dream” se...,('Last February Brandon John joined more than...,('When the offered season ticket holders the ...,('They were immigrants from Iraq and India th...,('Rain crashes down on onto the tarp thunder ...,,('It’s been almost a full month since Thomas M...
1,1,('A woman’s toe was bitten off during a fight ...,('Dr Kenneth C Edelin whose historic 1975 m...,('In what police described as a horrific trage...,('nOn the weekend before he was to be sworn in...,('n n n and are expanding their burger bus...,('One bone chilling Saturday night about 11 mo...,('The Federal Reserve must take a “patient app...,('n and Michelle Wu stood side by side at the ...,('MEMPHIS — The thank yous were almost endless...,...,('As neighbors living a world apart they awok...,('The 26 year old Chinese entrepreneur had jus...,('n n for help The harrowing leaps from smoky...,('nis here the nascent organizing committee...,('Don’t ask how the sausage is made the old s...,('As America’s most Irish American city it co...,('Beth Israel Deaconess Medical Center said We...,,,('Mayor Martin J Walsh’s pick for acting fire...
2,2,('It’s the ideal time for a tech journalist to...,('When they go up they form the centerpiece o...,('The fatal shooting today of a 9 year old Mat...,('Boston saw its first two homicides of 2014 o...,('Mayor Martin J Walsh plans to set aside spa...,('nhe Boston Public Library is using three dim...,('ncooler buses For decades economists like ...,('Jack Conley who grew up in South Boston and...,('Marty Walsh has turned the casino licensing ...,...,('When Boston firefighters pulled up to a vaca...,('It was just some children playing baseball o...,('n n for help The harrowing leaps from smoky...,('nis here the nascent organizing committee...,('Former Boston councilor Felix D Arroyo the...,('In nine days Wegmans will open a new superm...,('The Rev Joseph Hung Duc Tran had been ordai...,,,('His cousin would come to Children’s Hospital...
3,3,('The first major snowstorm of 2014 has intens...,('Good Morning Chief Justice Ireland Presiden...,('During a highly publicized Boston Police Dep...,('An elderly woman became Boston’s first fire ...,('By last Christmas Frank Nunes seemed to hav...,('QUINCY — Massachusetts Democrats saw a new B...,('When Ari S Heckman was growing up in Provid...,('Mayor Martin J Walsh said Wednesday that he...,('Marty Walsh has turned the casino licensing ...,...,('Two Charlestown teenagers — a 14 year old gi...,('Boston police are asking for the public’s he...,('n n for help The harrowing leaps from smoky...,('The first thing everyone noticed about Dawnn...,('nn the middle of evening rush hour at South ...,('n n n nThe 150 dancing singing and drummin...,('nLittle about the ordinary looking two famil...,,,('The is offering 15 percent off purchases fr...
4,4,('One Fund Boston which initially collected $...,('Good Morning Chief Justice Ireland Presiden...,('At Greater Love Tabernacle Church today mor...,('n officials in Boston want to give 4 500 mid...,('SALEM — Martha Coakley had already made the ...,('The state’s highest court has thrown out the...,('Marty Walsh had never met 

In [9]:
def parse_value(x):
    try:
        return literal_eval(x)   
    except Exception as e:
        return None

def parse_column_values(data_frame):
    for i in range(1, len(data_frame.columns)):
        column_name = data_frame.columns[i]
        data_frame[column_name] = data_frame[column_name].apply(lambda x: parse_value(x))   
    return data_frame.iloc[: , 1:]

In [10]:
def tokenize(text):
    if text is None:
        return None

    text = text[0]
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text.lower())
    tokens = []
    for word in nltk.word_tokenize(text):
        if word in all_stopwords:
            continue
        tokens.append(word)

    return " ".join(tokens)

In [11]:
def generate_training_test_doc_set(data_frame, train_column):
    training_values = []
    test_values = []
    for i in range(len(data_frame.columns)):
        column_name = data_frame.columns[i]
        if (column_name == train_column):
            train_val = data_frame[column_name].apply(lambda x: tokenize(x))
            train_val = list(filter(None.__ne__, train_val))
            training_values.extend(train_val)
        else:
            test_val = data_frame[column_name].apply(lambda x: tokenize(x))
            test_val = list(filter(None.__ne__, test_val))
            test_values.extend(test_val)

    return (training_values, test_values)

In [24]:
def generate_frequency_matrix(year, file_name):
  
    important_neighborhood_words = {}

    df = pd.read_csv(f'{CONTENT_ROOT_PATH}/{file_name}')
    parsed_df = parse_column_values(df)

    print(f"generating tf-idf matrix for {year} dataset")
  
    for i in range(len(parsed_df.columns)):
        column_name = parsed_df.columns[i]
        (training_values, test_values) = generate_training_test_doc_set(parsed_df, column_name)

        if len(training_values) == 0 or len(test_values) == 0:
            continue

        tfidf_vectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english', use_idf=True)

        tfidf_vectorizer.fit(training_values)
        tfidf_train = tfidf_vectorizer.transform(training_values)
        tfidf_term_vectors  = tfidf_vectorizer.transform(test_values)

        feature_names = np.array(tfidf_vectorizer.get_feature_names())
        
        df = pd.DataFrame(tfidf_term_vectors.toarray(), columns = feature_names)
        df.to_csv(f'../data/processed/TF-IDF/{year}/tf-idf-{column_name}', index=False)

    return (tfidf_term_vectors, feature_names)

In [25]:
top_neighborhood_words = generate_frequency_matrix(2014, data_file_name_dict['2014'])

generating tf-idf matrix for 2014 dataset


In [26]:
top_neighborhood_words = generate_frequency_matrix(2015, data_file_name_dict['2015'])

generating tf-idf matrix for 2015 dataset


In [27]:
top_neighborhood_words = generate_frequency_matrix(2016, data_file_name_dict['2016'])

generating tf-idf matrix for 2016 dataset


In [28]:
top_neighborhood_words = generate_frequency_matrix(2017, data_file_name_dict['2017'])

generating tf-idf matrix for 2017 dataset


In [29]:
top_neighborhood_words = generate_frequency_matrix(2018, data_file_name_dict['2018'])

generating tf-idf matrix for 2018 dataset
